<a href="https://colab.research.google.com/github/qweliant/DS-Unit-2-Kaggle-Challenge/blob/master/module3/assignment_kaggle_challenge_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 3


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.

## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### More Categorical Encodings

**1.** The article **[Categorical Features and Encoding in Decision Trees](https://medium.com/data-design/visiting-categorical-features-and-encoding-in-decision-trees-53400fa65931)** mentions 4 encodings:

- **"Categorical Encoding":** This means using the raw categorical values as-is, not encoded. Scikit-learn doesn't support this, but some tree algorithm implementations do. For example, [Catboost](https://catboost.ai/), or R's [rpart](https://cran.r-project.org/web/packages/rpart/index.html) package.
- **Numeric Encoding:** Synonymous with Label Encoding, or "Ordinal" Encoding with random order. We can use [category_encoders.OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html).
- **One-Hot Encoding:** We can use [category_encoders.OneHotEncoder](http://contrib.scikit-learn.org/categorical-encoding/onehot.html).
- **Binary Encoding:** We can use [category_encoders.BinaryEncoder](http://contrib.scikit-learn.org/categorical-encoding/binary.html).


**2.** The short video 
**[Coursera — How to Win a Data Science Competition: Learn from Top Kagglers — Concept of mean encoding](https://www.coursera.org/lecture/competitive-data-science/concept-of-mean-encoding-b5Gxv)** introduces an interesting idea: use both X _and_ y to encode categoricals.

Category Encoders has multiple implementations of this general concept:

- [CatBoost Encoder](http://contrib.scikit-learn.org/categorical-encoding/catboost.html)
- [James-Stein Encoder](http://contrib.scikit-learn.org/categorical-encoding/jamesstein.html)
- [Leave One Out](http://contrib.scikit-learn.org/categorical-encoding/leaveoneout.html)
- [M-estimate](http://contrib.scikit-learn.org/categorical-encoding/mestimate.html)
- [Target Encoder](http://contrib.scikit-learn.org/categorical-encoding/targetencoder.html)
- [Weight of Evidence](http://contrib.scikit-learn.org/categorical-encoding/woe.html)

Category Encoder's mean encoding implementations work for regression problems or binary classification problems. 

For multi-class classification problems, you will need to temporarily reformulate it as binary classification. For example:

```python
encoder = ce.TargetEncoder(min_samples_leaf=..., smoothing=...) # Both parameters > 1 to avoid overfitting
X_train_encoded = encoder.fit_transform(X_train, y_train=='functional')
X_val_encoded = encoder.transform(X_train, y_val=='functional')
```

**3.** The **[dirty_cat](https://dirty-cat.github.io/stable/)** library has a Target Encoder implementation that works with multi-class classification.

```python
 dirty_cat.TargetEncoder(clf_type='multiclass-clf')
```
It also implements an interesting idea called ["Similarity Encoder" for dirty categories](https://www.slideshare.net/GaelVaroquaux/machine-learning-on-non-curated-data-154905090).

However, it seems like dirty_cat doesn't handle missing values or unknown categories as well as category_encoders does. And you may need to use it with one column at a time, instead of with your whole dataframe.

**4. [Embeddings](https://www.kaggle.com/learn/embeddings)** can work well with sparse / high cardinality categoricals.

_**I hope it’s not too frustrating or confusing that there’s not one “canonical” way to encode categorcals. It’s an active area of research and experimentation! Maybe you can make your own contributions!**_

### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module3')

Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
From https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge
 * branch            master     -> FETCH_HEAD
Already up to date.


In [0]:
import pandas as pd
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_regression, SelectKBest, f_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
import numpy as np

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv('../data/waterpumps/train_features.csv'), 
                 pd.read_csv('../data/waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv('../data/waterpumps/test_features.csv')
sample_submission = pd.read_csv('../data/waterpumps/sample_submission.csv')

In [0]:
def wrangle(X):
    X = X.copy()
    # # Get a list of the top 10 neighborhoods
    top10 = X['wpt_name'].value_counts()[:10].index
    X.loc[~X['wpt_name'].isin(top10), 'wpt_name'] = 'other'

    top10 = X['subvillage'].value_counts()[:10].index
    X.loc[~X['subvillage'].isin(top10), 'subvillage'] = 'other'

    top10 = X['region'].value_counts()[:10].index
    X.loc[~X['region'].isin(top10), 'region'] = 'other'

    # top10 = X['lga'].value_counts()[:10].index
    # X.loc[~X['lga'].isin(top10), 'lga'] = 'other'

    top10 = X['ward'].value_counts()[:10].index
    X.loc[~X['ward'].isin(top10), 'ward'] = 'other'

    # top10 = X['basin'].value_counts()[:10].index
    # X.loc[~X['basin'].isin(top10), 'basin'] = 'other'

    top10 = X['funder'].value_counts()[:10].index
    X.loc[~X['funder'].isin(top10), 'funder'] = 'other'

    top10 = X['scheme_name'].value_counts()[:10].index
    X.loc[~X['scheme_name'].isin(top10), 'scheme_name'] = 'other'

    top10 = X['installer'].value_counts()[:10].index
    X.loc[~X['installer'].isin(top10), 'installer'] = 'other'

    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    cols_with_zeros = ['longitude', 'latitude', 'construction_year', 'gps_height', 'population']
    X['construction_year'] = X['construction_year'].astype(dtype='int32')
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        # X[col+'_MISSING'] = X[col].isnull()
    X = X.drop(columns=['quantity_group', 'payment_type'])

    
    X.date_recorded = pd.to_datetime(X.date_recorded)
    X =  X.set_index('date_recorded')
    
    X['inspec_year'] = X.index.year.astype(dtype='int32')
    X['time_before_inspection'] = (X.inspec_year - X.construction_year)
    X['years_MISSING'] = X['time_before_inspection'].isnull() 
    return(X)


def train_validate(df):
    train_set, validation_set = train_test_split(df, train_size=0.80, test_size=0.20, stratify=train['status_group'], random_state=42, )
    return train_set, validation_set

In [0]:
test.replace({'functional':0, 'functional needs repair':1, 'non functional':2}, inplace=True)

In [0]:
train_set, validation_set = train_validate(train)

In [0]:
train_set = wrangle(train_set)
validation_set = wrangle(validation_set)
test = wrangle(test)

In [0]:
target = "status_group"
numbers_columns = train_set.select_dtypes(include='number').columns.drop('id').tolist()
categorical_columns = train_set.select_dtypes(exclude='number').columns.drop(target).tolist()
drop_high_cardinality = [col for col in categorical_columns if train_set[col].nunique() <= 200]
features = numbers_columns + drop_high_cardinality

In [0]:
X_train = train_set[features]
y_train = train_set[target]
X_val = validation_set[features]
y_val = validation_set[target]
X_test = test[features]

In [9]:
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'), 
    RandomForestClassifier(n_estimators=100, random_state=2121, n_jobs=-1)
)

pipeline.fit(X_train, y_train)


print('Train Accuracy:', pipeline.score(X_train, y_train))
print('Validation Accuracy:', pipeline.score(X_val, y_val))

Train Accuracy: 0.9970749158249158
Validation Accuracy: 0.8085016835016835


In [11]:
stack_cup = {
    'randomforestclassifier__max_features': range(1,20),
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions = stack_cup,
    n_iter=100,
    cv=5,
    verbose=10,
    return_train_score=True,
    n_jobs=-1,
)

search.fit(X_train, y_train)

Fitting 5 folds for each of 19 candidates, totalling 95 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 19 is smaller than n_iter=100. Running 19 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   12.2s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   22.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   53.3s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:  9.8min
[Parallel(n_jobs=-1)]: Done  81

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=['funder',
                                                                   'installer',
                                                                   'wpt_name',
                                                                   'basin',
                                                                   'subvillage',
                                                                   'region',
                                                                   'lga',
                                                                   'ward',
                                                                   'public_meeting',
                                                                   'recorded_by',
                                         

In [23]:
# train_set.date_recorded = pd.to_datetime(train_set.date_recorded)
# validation_set.date_recorded = pd.to_datetime(validation_set.date_recorded)
test_test = test.copy()
y_pred = search.predict(X_test)
submission = test[['id']]
submission['status_group'] = y_pred
submission.replace({0:'functional', 1:'functional needs repair', 2:'non functional'}, inplace=True)
submission.head()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


,id,status_group
date_recorded,,
2013-02-04,50785,functional
2013-02-04,51630,functional
2013-02-01,17168,functional
2013-01-22,45559,non functional
2013-03-27,49871,functional


In [0]:
submission.to_csv('submission.csv', index=False)

In [0]:
from google.colab import files

files.download('submission.csv')